# Generic Rule Induction Notebook

Continuoulsy refined.

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment

### Configuration


In [144]:
CONFIG1 = { 
    'NAME' : 'binary-churn-quantile-brcg', 
    'DATA_SET': '../data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'BRCG',
    'TARGET_LABEL': 'CHURN',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': 'T'
     }
CONFIG2 = { 
    'NAME' : 'continuous-churn', 
    'DATA_SET': 'data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'CHURN', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'pChurn',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': None
     }
CONFIG3 = { 
    'NAME' : 'bike-demand', 
    'DATA_SET': '../data/SeoulBikeData.csv',
    'DATA_TYPES': {'Rented Bike Count': float, 'Hour': float, 'Humidity': float, 'Visibility (10m)': float, 'RatePlan': str},
    'DROP': ['Date'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Rented Bike Count',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Dew point temperature(C)',
    'POS_CLASS': None
     }
CONFIG4 = { 
    'NAME' : 'heloc', 
    'DATA_SET': '../data/heloc.csv',
    'DATA_TYPES': {},
    'DROP': ['RiskPerformance'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Probabilities',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'ExternalRiskEstimate',
    'POS_CLASS': None
     }

CONFIG = CONFIG3
print('Proceed with configuration:', CONFIG['NAME'])

Proceed with configuration: bike-demand


In [145]:
import pandas as pd
import numpy as np
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
# from ripper.Ripper import Ripper
import time
import warnings

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
from aix360.algorithms.rbm import FeatureBinarizer, FeatureBinarizerFromTrees

# from explainer import Explainer

# TODO make imports dependent on CONFIG
# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [146]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG':
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert) 
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rented Bike Count         8760 non-null   float64
 1   Hour                      8760 non-null   float64
 2   Temperature(C)            8760 non-null   float64
 3   Humidity(%)               8760 non-null   int64  
 4   Wind speed (m/s)          8760 non-null   float64
 5   Visibility (10m)          8760 non-null   float64
 6   Dew point temperature(C)  8760 non-null   float64
 7   Solar Radiation (MJ/m2)   8760 non-null   float64
 8   Rainfall(mm)              8760 non-null   float64
 9   Snowfall (cm)             8760 non-null   float64
 10  Seasons                   8760 non-null   object 
 11  Holiday                   8760 non-null   object 
 12  Functioning Day           8760 non-null   object 
dtypes: float64(9), int64(1), object(3)
memory usage: 889.8+ KB


,Rented Bike Count,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,254.0,0.0,-5.2,37,2.2,2000.0,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,204.0,1.0,-5.5,38,0.8,2000.0,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,173.0,2.0,-6.0,39,1.0,2000.0,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,107.0,3.0,-6.2,40,0.9,2000.0,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,78.0,4.0,-6.0,36,2.3,2000.0,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [147]:
df[CONFIG['TARGET_LABEL']].describe()

count    8760.000000
mean      704.602055
std       644.997468
min         0.000000
25%       191.000000
50%       504.500000
75%      1065.250000
max      3556.000000
Name: Rented Bike Count, dtype: float64

### Train, Test Split

In [148]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (6132, 12) (6132,)
Test: (2628, 12) (2628,)


### Reference Performance

In [149]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

needs prior encoding of categoricals


### Binarization

In [150]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=True) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6132 entries, 1444 to 7270
Columns: 134 entries, ('Hour', '<=', 2.0) to ('Functioning Day', 'not', '')
dtypes: int64(134)
memory usage: 6.3 MB


operation     <=                                                          \
value     -15.10 -7.30  -2.10   1.34   5.20   8.90   13.00  16.60  21.00   
1444           1      1      1      1      1      1      1      1      1   
1652           1      1      1      1      1      1      1      1      1   
7496           0      0      0      0      1      1      1      1      1   
1893           0      1      1      1      1      1      1      1      1   
3880           0      0      0      0      0      0      1      1      1   
2406           0      0      0      1      1      1      1      1      1   
5082           0      0      0      0      0      0      0      0      0   
8553           1      1      1      1      1      1      1      1      1   
4663           0      0      0      0      0      0      0      1      1   
1966           1      1      1      1      1      1      1      1      1   

operation      >                                                          
value     -15.10 -7.30  -2.10   1.34   5.20   8.90   13.00  16.60  21.00  
1444           0      0      0      0      0      0      0      0      0  
1652           0      0      0      0      0      0      0      0      0  
7496           1      1      1      1      0      0      0      0      0  
1893           1      0      0      0      0      0      0      0      0  
3880           1      1      1      1      1      1      0      0      0  
2406           1      1      1      0      0      0      0      0      0  
5082           1      1      1      1      1      1      1      1      1  
8553           0      0      0      0      0      0      0      0      0  
4663           1      1      1      1      1      1      1      0      0  
1966           0      0      0      0      0      0      0      0      0

### Rule Induction

In [151]:
start_time = time.time()

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    print('TODO plugin RIPPER here')
    # Ripper(d=2, k=2, pruning_threshold=50, mode='python')
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('training time: ' + str(end_time - start_time))

training time: 10.453630924224854


### Evaluation

In [152]:
if CONFIG['TYPE'] == 'BINARY':
    # compute predictive accuracy
    # print(estimator.dnf_rules.items())
    y_pred = estimator.predict(x_test_bin)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy)
    # print("Accuracy: %.2f%%" % (accuracy * 100.0))
    # print('Accuracy 2:', accuracy_score(y_test, estimator.predict(x_test)))
    # print('Accuracy 3', estimator.score(x_test, y_test))
    # print('Accuracy 4', estimator.score(x_test, y_test))

    # res = estimator.score(x_test, y_test) # this score function is defect
    # print('acc: ' + str(res))
    # print('rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
    # estimator.rule_map
    # print(estimator.rule_list_to_pretty_string())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')


R2 Score = 0.7268003461209963
Explained Variance = 0.7268027536260733
Mean abs. error = 231.308866960986
Max error = 1945.854933920119


In [153]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)

# uncomment the following line for a full optimized view of the model as data frame
# explanation.style

                                                 rule coefficient
0                                         (intercept)     240.167
1   Hour > 14.00 AND Dew point temperature(C) > -1...      575.35
2   Hour > 7.00 AND Humidity(%) <= 86.00 AND Rainf...     508.038
3   Hour <= 21.00 AND Dew point temperature(C) <= ...     424.983
4   Hour > 16.00 AND Temperature(C) > -3.50 AND Ra...      423.04
5   Temperature(C) > 9.80 AND Humidity(%) <= 86.00...     297.764
6   Temperature(C) > 13.90 AND Rainfall(mm) <= 0.0...     207.013
7                             Temperature(C) <= 21.00    -175.424
8             Seasons == Autumn AND Functioning Day       161.358
9                  Hour > 16.00 AND Functioning Day       147.418
10  Temperature(C) > 18.00 AND Rainfall(mm) <= 0.0...     146.568
11  Dew point temperature(C) > -15.10 AND Solar Ra...     134.243
12    Hour > 2.00 AND Solar Radiation (MJ/m2) <= 0.00    -133.461
13                      Hour <= 16.00 AND Hour > 2.00    -131.342
14  Solar 